In [23]:
import puffin.tardis_importer as ti
symbol = 'BTC'
spot_trades = ti.generate_trade_data_from_gzip(f'{symbol}/USD', f'./datasets/ftx_trades_2021-10-28_{symbol}-USD.csv.gz')
spot_tobs = ti.generate_tob_data_from_gzip(f'{symbol}/USD', f'./datasets/ftx_quotes_2021-10-28_{symbol}-USD.csv.gz')
future_trades = ti.generate_trade_data_from_gzip(f'{symbol}/USD', f'./datasets/ftx_trades_2021-10-28_{symbol}-PERP.csv.gz')
future_tobs = ti.generate_tob_data_from_gzip(f'{symbol}/USD', f'./datasets/ftx_quotes_2021-10-28_{symbol}-PERP.csv.gz')

In [34]:
spot_tobs['timestamp'] = spot_tobs['exchange_timestamp_nanos'] // 1000000000
spot_tobs = spot_tobs.drop_duplicates(subset='timestamp', keep='last')
spot_trades['timestamp'] = spot_trades['exchange_timestamp_nanos'] // 1000000000
givens = spot_trades[spot_trades['given']]
givens = givens[['timestamp', 'price']].groupby(by=['timestamp']).min()
givens = givens.rename(columns={'price': 'given_price'})
paids = spot_trades[~spot_trades['given']]
paids = paids[['timestamp', 'price']].groupby(by=['timestamp']).max()
paids = paids.rename(columns={'price': 'paid_price'})
bids = spot_tobs[['timestamp', 'bid_price']]
bids = bids.set_index('timestamp')
asks = spot_tobs[['timestamp', 'ask_price']]
asks = asks.set_index('timestamp')

In [5]:
import pandas as pd
left = pd.merge(bids, givens, how='left', left_index=True, right_index=True)
left['how_deep'] = left['bid_price'] - left['price']
left = left['how_deep'].value_counts(dropna=False)
left = left.sort_index(ascending=False).to_frame().cumsum()
left['how_deep'] = left['how_deep'].astype('float')
left['how_deep'] = left['how_deep'].div(left['how_deep'].iloc[-1])
left = left[left.index >= 0]
left

 NaN      55725
 0.0      13158
-1.0       1481
-2.0        910
-3.0        900
          ...  
 199.0        1
 93.0         1
 104.0        1
 85.0         1
 213.0        1
Name: how_deep, Length: 166, dtype: int64

In [4]:
import pandas as pd
right = pd.merge(asks, paids, how='left', left_index=True, right_index=True)
right['how_deep'] = right['price'] - right['ask_price']
right = right['how_deep'].value_counts(dropna=False)
right = right.sort_index(ascending=False).to_frame().cumsum()
right['how_deep'] = right['how_deep'].astype('float')
right['how_deep'] = right['how_deep'].div(right['how_deep'].iloc[-1])
right = right[right.index >= 0]
right

,how_deep
809.0,0.000012
345.0,0.000024
333.0,0.000037
310.0,0.000049
254.0,0.000061
...,...
4.0,0.036327
3.0,0.042039
2.0,0.048371
1.0,0.058162


In [37]:
both_sides = spot_tobs[['timestamp', 'bid_price', 'ask_price']]
both_sides = both_sides.set_index('timestamp')
both_sides = pd.merge(both_sides, givens, how='left', left_index=True, right_index=True)
both_sides = pd.merge(both_sides, paids, how='left', left_index=True, right_index=True)
both_sides[['given_price', 'paid_price']]

,given_price,paid_price
timestamp,,
1635379200,NaN,NaN
1635379201,58449.0,NaN
1635379202,NaN,58467.0
1635379203,NaN,58457.0
1635379204,NaN,58458.0
...,...,...
1635465595,NaN,NaN
1635465596,NaN,NaN
1635465597,NaN,NaN


In [38]:
both_sides[['given_price', 'paid_price']].dropna(how='all')

,given_price,paid_price
timestamp,,
1635379201,58449.0,NaN
1635379202,NaN,58467.0
1635379203,NaN,58457.0
1635379204,NaN,58458.0
1635379205,NaN,58465.0
...,...,...
1635465582,NaN,60618.0
1635465583,NaN,60621.0
1635465585,NaN,60621.0


In [41]:
test


0         1635379201135023000
1         1635379201135690000
2         1635379202039265000
3         1635379202885625000
4         1635379202885625000
                 ...         
215747    1635465599281121000
215748    1635465599281121000
215749    1635465599281121000
215750    1635465599775669000
215751    1635465599901175000
Name: exchange_timestamp_nanos, Length: 215752, dtype: int64